In [1]:
##Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import seaborn as sns
import warnings
import matplotlib.pyplot as plt
from astropy.stats import bayesian_blocks
warnings.filterwarnings('ignore')
sns.set(rc={'figure.figsize':(200,5)})
%matplotlib inline

DATA ACCESS

In [2]:
dataset = pd.read_excel('Hackathon Price data31696da.xlsx')

In [3]:
dataset.head()

,Mcat Name,Subcat name,PC_ITEM_ID,PC_ITEM_NAME,FK_IM_SPEC_MASTER_DESC,FK_IM_SPEC_OPTIONS_DESC,PC_ITEM_FOB_PRICE,PC_ITEM_MOQ_UNIT_TYPE
0,HMI Touch Panel,"AC Drives, Recorders, Instrumentation Equipmen...",1,Touch Panel,Display Size,"15.6"" to 80 """,5500.0,Piece
1,HMI Touch Panel,"AC Drives, Recorders, Instrumentation Equipmen...",2,Human Machine Interface,Type,Fully Automatic,4500.0,Unit
2,HMI Touch Panel,"AC Drives, Recorders, Instrumentation Equipmen...",3,Mitsubishi HMI,Display Type,TFT,12000.0,Unit
3,HMI Touch Panel,"AC Drives, Recorders, Instrumentation Equipmen...",3,Mitsubishi HMI,Model Number/Name,GOT,12000.0,Unit
4,HMI Touch Panel,"AC Drives, Recorders, Instrumentation Equipmen...",3,Mitsubishi HMI,Product Type,HMI,12000.0,Unit


In [4]:
dataset.columns

Index(['Mcat Name', 'Subcat name', 'PC_ITEM_ID', 'PC_ITEM_NAME',
       'FK_IM_SPEC_MASTER_DESC', 'FK_IM_SPEC_OPTIONS_DESC',
       'PC_ITEM_FOB_PRICE', 'PC_ITEM_MOQ_UNIT_TYPE'],
      dtype='object')

In [5]:
data = dataset[['Mcat Name','FK_IM_SPEC_MASTER_DESC', 'FK_IM_SPEC_OPTIONS_DESC',
       'PC_ITEM_FOB_PRICE', 'PC_ITEM_MOQ_UNIT_TYPE']]

In [6]:
data.head()

,Mcat Name,FK_IM_SPEC_MASTER_DESC,FK_IM_SPEC_OPTIONS_DESC,PC_ITEM_FOB_PRICE,PC_ITEM_MOQ_UNIT_TYPE
0,HMI Touch Panel,Display Size,"15.6"" to 80 """,5500.0,Piece
1,HMI Touch Panel,Type,Fully Automatic,4500.0,Unit
2,HMI Touch Panel,Display Type,TFT,12000.0,Unit
3,HMI Touch Panel,Model Number/Name,GOT,12000.0,Unit
4,HMI Touch Panel,Product Type,HMI,12000.0,Unit


**DATA CLEANING**

In [7]:
cleaning_column={'piece':'piece', 'unit':'unit', 'per piece':'piece', 'number':'number', 'nos':'number', 'person':'person', 'box':'box',
       'piece(s)':'piece', '1 unit':'unit', 'set':'set', 'no': 'number', 'square meter':'square meter', 'bag':'bag', 'pair':'pair',
       'litre':'litre', 'litre':'litre', "20' container":"20 container", 'barrel':'barrel', '1pc':'piece', 'month':'month',
       '1 pc':'piece', 'kit':'kit', 'no(s)':'piece', 'kilogram':'kilogram', 'kilogram(s)':'kilogram', 'meter':'meter', 'kg':'kilogram',
       'roll':'roll', 'one':'one', 'unit(s)':'unit', 'feet':'feet', '1 set':'set', 'foot':'foot', 'square feet':'square feet',
       'running feet':'running feet', 'units':'unit', 'packet':'packet', 'per feet':'feet', 'mtr':'meter', 'per mtrs':'meter',
       'plate':'plate', 'micrometer':'micrometer', 'centimeter':'centimeter', 'running meter':'running meter', 'meter(s)':'meter',
       'bundle':'bundle', 'pieces':'piece', 'peice':'peice', '1 piece':'piece', 'per box':'box',
       'per square feet':'square feet', 'per sq.ft.':'square feet', 'dozen':'dozen', 'tons':'tons', 'kilometer':'kilometer',
       'bottle':'bottle', 'kilogram onwards':'kilogram', 'pc_onwards':'piece', 'pouch':'pouch',
       'piece onwards':'piece', 'sheet':'sheet', 'carton':'carton', 'pack':'pack', 'millimeter':'millimeter', 'ounce':'ounce',
       'square feet(s)':'square feet', 'square inch':'square inch', 'hour':'hour', 'day':'day', 'per unit':'unit', 'each':'unit',
       'numbers':'number', 'inch':'inch', 'milliliter':'milliliter', 'ton':'ton', '1 ltr':'litre', 'metric ton':'metric ton',
       '50kg':'50kg', 'gram':'gram', '1 liter':'litre', '1 box':'box', 'long ton':'long ton', 'long ton':'long ton', 'per kg':'kilogram',
       'unit onward':'unit', 'pcs':'piece', 'kilowatt':'kilowatt', 'cubic feet':'cubic feet', 'square fee':'square feet',
       'per nos':'number', 'watt':'watt', 'per pcs':'piece','liter':'litre','gallon':'gallon'}

In [8]:
data['PC_ITEM_MOQ_UNIT_TYPE']=data['PC_ITEM_MOQ_UNIT_TYPE'].apply(lambda x:x.lower())

In [9]:
data['PC_ITEM_MOQ_UNIT_TYPE']=data['PC_ITEM_MOQ_UNIT_TYPE'].apply(lambda x:cleaning_column[x])

In [10]:
# data.to_excel('clean.xlsx')

**DATA DRILLING FOR BETTER INSIGHT**

***PART -1(MCAT - Unit wise Min-Max price)***

In [11]:
data.columns

Index(['Mcat Name', 'FK_IM_SPEC_MASTER_DESC', 'FK_IM_SPEC_OPTIONS_DESC',
       'PC_ITEM_FOB_PRICE', 'PC_ITEM_MOQ_UNIT_TYPE'],
      dtype='object')

In [12]:
mcat_name_list = ['HMI Touch Panel', 'AC Motors', 'Single Phase Induction Motors',
       'Three Phase Induction Motor', 'Refrigeration Compressors',
       'Compressor Oil', 'Desiccant Dryers', 'Rotary Screw Compressors',
       'High Pressure Air Compressor', 'Tractor Mounted Air Compressor',
       'Single Stage Reciprocating Compressor', 'Diesel Air Compressor',
       'Pneumatic Impact Wrenches', 'Air Spray Gun', 'Fleece Fabrics',
       'Lining Fabric', 'Audio Amplifiers', 'Belt Conveyors',
       'Conveyor Chains', 'Aluminum Cables', 'Optical Fiber',
       'Roller Conveyor', 'Rubber Conveyor Belt', 'PVC Cables',
       'Slat Conveyor', 'Copper Cables', 'Stainless Steel Cable Tie',
       'Heat Resistant Conveyor Belt', 'Camera Lenses',
       'Sony Digital Camera', 'Canon Digital Camera', 'Porcelain Tiles',
       'CNC Milling Machine', 'Drum Mixers', 'Leather Flash Drive',
       'Knitted Yarn', 'Polyester Thread', 'Spun Yarn', 'Textile Yarn',
       'Pouch Filling Machine', 'Powder Filling Machine',
       'Tube Filling Machine', 'Twisted Yarn', 'Bottle Filling Machines',
       'Polyester Filament Yarn', 'Roti Makers', 'Liquid Filling Machine',
       'Jar Filling Machine', 'Polyester Textured Yarn',
       'Wire Drawing Machines', 'Bag Filling Machine', 'Coffee Makers',
       'Electric Irons', 'Food Processor', 'Steam Iron',
       'Vertical Form Fill Seal Machines', 'Hydraulic Bending Machine',
       'Toaster', 'Tea Packaging Machine', 'Granules Packing Machine',
       'Water Packaging Machine', 'Dyed Polyester Yarn',
       'Juicer Mixer Grinder', 'Hand Operated Hydraulic Press',
       'Mineral Water Filling Machine', 'Tractor Front End Loader',
       'Acme Safety Shoes', 'Air Conditioner Voltage Stabilizer',
       'Aluminium False Ceilings', 'Aluminium Melting Furnace',
       'Bench Drilling Machine', 'Biometric Devices', 'Backhoe Loader',
       'Skid Steer Loader', 'Wheel Loader', 'End Loader',
       'Bucket Elevator', 'Busbar Systems', 'Butt Hinges',
       'Button Hole Machine', 'Color Printers', 'Colored Contact Lenses',
       'Copper Earthing Electrode', 'Ladies Palazzo', 'Bio Fertilizers',
       'Marble God Statue', 'Electric Industrial Oven', 'Belt Pulley',
       'Womens Capri', 'Mens Capri', 'Solar Pump Controller',
       'Embroidery Machine', 'Wooden Crates',
       'Hair Removal Laser Machine', 'PVC Roofing Sheet',
       'Water Demineralization Plant', 'Solar Fans', 'Fire Safety Wear',
       'Jindal GI Pipe', 'Eco Friendly Disposable Plates']

In [13]:
unit_type_list = ['piece', 'unit', 'number', 'person', 'box', 'set', 'square meter',
       'bag', 'pair', 'litre', '20 container', 'barrel', 'month', 'kit',
       'kilogram', 'meter', 'roll', 'one', 'feet', 'foot', 'square feet',
       'running feet', 'packet', 'plate', 'micrometer', 'centimeter',
       'running meter', 'bundle', 'peice', 'dozen', 'tons', 'kilometer',
       'bottle', 'pouch', 'sheet', 'carton', 'pack', 'millimeter',
       'ounce', 'square inch', 'hour', 'day', 'inch', 'milliliter', 'ton',
       'metric ton', '50kg', 'gram', 'long ton', 'gallon', 'kilowatt',
       'cubic feet', 'watt']

In [14]:
data[(data['Mcat Name']=='HMI Touch Panel')&(data['PC_ITEM_MOQ_UNIT_TYPE']=='unit')]

,Mcat Name,FK_IM_SPEC_MASTER_DESC,FK_IM_SPEC_OPTIONS_DESC,PC_ITEM_FOB_PRICE,PC_ITEM_MOQ_UNIT_TYPE
1,HMI Touch Panel,Type,Fully Automatic,4500.0,unit
2,HMI Touch Panel,Display Type,TFT,12000.0,unit
3,HMI Touch Panel,Model Number/Name,GOT,12000.0,unit
4,HMI Touch Panel,Product Type,HMI,12000.0,unit
5,HMI Touch Panel,Brand,Mitsubishi,12000.0,unit
6,HMI Touch Panel,Size,"5',7.5',10,15",12000.0,unit
7,HMI Touch Panel,Screen Size,"5,7,10,15",12000.0,unit
8,HMI Touch Panel,Display Type,Digital,10000.0,unit
9,HMI Touch Panel,Type,Fully Automatic,10000.0,unit
10,HMI Touch Panel,Operating System,Windows 8,10000.0,unit


In [15]:
@interact
def plot_data(mcat_name=mcat_name_list,Graph_type=['strip','box']):
    x=data[(data['Mcat Name']==mcat_name)]
    d=sns.catplot(x='PC_ITEM_MOQ_UNIT_TYPE',y='PC_ITEM_FOB_PRICE',data=x,kind=Graph_type,aspect=3)
    return data[(data['Mcat Name']==mcat_name)]

interactive(children=(Dropdown(description='mcat_name', options=('HMI Touch Panel', 'AC Motors', 'Single Phase…

In [16]:
data_1 = data[['Mcat Name', 'PC_ITEM_MOQ_UNIT_TYPE','PC_ITEM_FOB_PRICE']]

In [17]:
'''
Ouput: dictionary 


'''
output={}
cleaned_price={}
##Creating Subset based on Categories
data1=data.groupby('Mcat Name')
for category,data_df in data1:
    ## loop through each category
    unit_df =data_df.groupby('PC_ITEM_MOQ_UNIT_TYPE')
    for unit_type,unit_df in unit_df:
        ##Loop through each unit
        if(len(unit_df)>2):
            #Clustering can be performed only on data set that has no. of records > no. of set cluster                   
            y_means = KMeans(n_clusters=2).fit(unit_df['PC_ITEM_FOB_PRICE'].values.reshape(-1,1)).labels_
            
            unit_df['predict']=y_means
            #non_anomaly list
            non_anomalies = []
            
            ##selecting the predominant cluster
            frequent_cluster_df=unit_df.groupby(['predict']).filter(lambda x: len(x)==unit_df.groupby(['predict']).size().max())
            #Calculating Standard deviation on the given subset
            random_data_std = np.std(frequent_cluster_df['PC_ITEM_FOB_PRICE'])
            #Calculating mean on the given subset
            random_data_mean = np.mean(frequent_cluster_df['PC_ITEM_FOB_PRICE'])
            anomaly_cut_off = random_data_std * 3
            #Lower limit: below 3 standard deviation from mean
            lower_limit  = random_data_mean - anomaly_cut_off 
            #Above 3 standard deviation from mean
            upper_limit = random_data_mean + anomaly_cut_off
            
            for outlier in frequent_cluster_df['PC_ITEM_FOB_PRICE']:
                if outlier > upper_limit or outlier < lower_limit:
                    pass
                else:
                    non_anomalies.append(outlier)
            
            #Dictionary with min and max for a given category and unit
            di_temp={'category':category,"unit_type":unit_type,"min_range":min(non_anomalies),"max_range":max(non_anomalies)}
            temp=category+'_'+unit_type
            output[temp]=di_temp
            cleaned_price[temp]=non_anomalies
        else:
            #For subset with no. of records less than 2
            di_temp={'category':category,"unit_type":unit_type,"min_range":min(unit_df['PC_ITEM_FOB_PRICE']),"max_range":max(unit_df['PC_ITEM_FOB_PRICE'])}
            temp=category+'_'+unit_type
            output[temp]=di_temp
            cleaned_price[temp]=non_anomalies

In [18]:
final_output_data=pd.DataFrame(output).T

In [19]:
final_output_data = final_output_data[['category','unit_type','min_range','max_range']]

In [20]:
final_output_data.to_csv('MCAT - Unit wise Min-Max price.csv')

In [21]:
final_output_data

,category,unit_type,min_range,max_range
AC Motors_box,AC Motors,box,1600,1600
AC Motors_number,AC Motors,number,22000,25000
AC Motors_piece,AC Motors,piece,45,12000
AC Motors_set,AC Motors,set,100,100
AC Motors_unit,AC Motors,unit,2000,28000
Acme Safety Shoes_box,Acme Safety Shoes,box,860,860
Acme Safety Shoes_pair,Acme Safety Shoes,pair,200,1100
Acme Safety Shoes_piece,Acme Safety Shoes,piece,200,399
Acme Safety Shoes_unit,Acme Safety Shoes,unit,680,720
Air Conditioner Voltage Stabilizer_box,Air Conditioner Voltage Stabilizer,box,300,1900


***PART -2(MCAT - Unit - ISQ --- Min Max price.)***

In [22]:
data[(data['Mcat Name']=='HMI Touch Panel')&(data['PC_ITEM_MOQ_UNIT_TYPE']=='unit')]

,Mcat Name,FK_IM_SPEC_MASTER_DESC,FK_IM_SPEC_OPTIONS_DESC,PC_ITEM_FOB_PRICE,PC_ITEM_MOQ_UNIT_TYPE
1,HMI Touch Panel,Type,Fully Automatic,4500.0,unit
2,HMI Touch Panel,Display Type,TFT,12000.0,unit
3,HMI Touch Panel,Model Number/Name,GOT,12000.0,unit
4,HMI Touch Panel,Product Type,HMI,12000.0,unit
5,HMI Touch Panel,Brand,Mitsubishi,12000.0,unit
6,HMI Touch Panel,Size,"5',7.5',10,15",12000.0,unit
7,HMI Touch Panel,Screen Size,"5,7,10,15",12000.0,unit
8,HMI Touch Panel,Display Type,Digital,10000.0,unit
9,HMI Touch Panel,Type,Fully Automatic,10000.0,unit
10,HMI Touch Panel,Operating System,Windows 8,10000.0,unit


In [23]:
@interact
def plot_data_f(mcat_name=mcat_name_list,unit_type=unit_type_list,graph_type=['strip','box']):
    x=data[(data['Mcat Name']==mcat_name)&(data['PC_ITEM_MOQ_UNIT_TYPE']==unit_type)]
    sns.catplot(x='FK_IM_SPEC_MASTER_DESC',y='PC_ITEM_FOB_PRICE',data=x,aspect=4,kind=graph_type)
    return x

interactive(children=(Dropdown(description='mcat_name', options=('HMI Touch Panel', 'AC Motors', 'Single Phase…

In [24]:
'''
Ouput: dictionary 


'''
output={}
##Creating Subset based on Categories
data2=data.groupby('Mcat Name')
for main_category, data1 in data2:
    data1=data1.groupby('PC_ITEM_MOQ_UNIT_TYPE')
    for category,data_df in data1:
        ## loop through each category
        unit_df =data_df.groupby('FK_IM_SPEC_MASTER_DESC')
        for unit_type,unit_df in unit_df:
            ##Loop through each unit
            if(len(unit_df)>2):
                #Clustering can be performed only on data set that has no. of records > no. of set cluster                   
                y_means = KMeans(n_clusters=2).fit(unit_df['PC_ITEM_FOB_PRICE'].values.reshape(-1,1)).labels_

                unit_df['predict']=y_means
                #non_anomaly list
                non_anomalies = []

                ##selecting the predominant cluster
                frequent_cluster_df=unit_df.groupby(['predict']).filter(lambda x: len(x)==unit_df.groupby(['predict']).size().max())
                #Calculating Standard deviation on the given subset
                random_data_std = np.std(frequent_cluster_df['PC_ITEM_FOB_PRICE'])
                #Calculating mean on the given subset
                random_data_mean = np.mean(frequent_cluster_df['PC_ITEM_FOB_PRICE'])
                anomaly_cut_off = random_data_std * 3
                #Lower limit: below 3 standard deviation from mean
                lower_limit  = random_data_mean - anomaly_cut_off 
                #Above 3 standard deviation from mean
                upper_limit = random_data_mean + anomaly_cut_off

                for outlier in frequent_cluster_df['PC_ITEM_FOB_PRICE']:
                    if outlier > upper_limit or outlier < lower_limit:
                        pass
                    else:
                        non_anomalies.append(outlier)
                #Dictionary with min and max for a given category and unit
                di_temp={'main_category':main_category,'unit_type':category,"sub_category":unit_type,"min_range":min(non_anomalies),"max_range":max(non_anomalies)}
                temp=main_category+'_'+category+'_'+unit_type
                output[temp]=di_temp

            else:
                #For subset with no. of records less than 2
                di_temp={'main_category':main_category,'unit_type':category,"sub_category":unit_type,"min_range":min(unit_df['PC_ITEM_FOB_PRICE']),"max_range":max(unit_df['PC_ITEM_FOB_PRICE'])}
                temp=main_category+'_'+category+'_'+unit_type
                output[temp]=di_temp

In [25]:
final_output_data_2=pd.DataFrame(output).T

In [26]:
final_output_data_2 = final_output_data_2[['main_category','unit_type','sub_category','max_range','min_range']]

In [27]:
final_output_data_2.to_csv('MCAT - Unit - ISQ --- Min Max price.csv')

In [28]:
final_output_data_2

,main_category,unit_type,sub_category,max_range,min_range
AC Motors_box_Power,AC Motors,box,Power,1600,1600
AC Motors_box_Speed,AC Motors,box,Speed,1600,1600
AC Motors_box_Voltage,AC Motors,box,Voltage,1600,1600
AC Motors_number_Brand,AC Motors,number,Brand,25000,25000
AC Motors_number_Motor Horsepower,AC Motors,number,Motor Horsepower,22000,22000
AC Motors_number_Mounting Type,AC Motors,number,Mounting Type,25000,25000
AC Motors_number_No of Phase,AC Motors,number,No of Phase,22000,5200
AC Motors_number_Phase,AC Motors,number,Phase,25000,25000
AC Motors_number_Power,AC Motors,number,Power,22000,22000
AC Motors_number_Speed,AC Motors,number,Speed,25000,22000


***PART -3(MCAT-Unit wise-Three most common prices.)***

In [66]:
@interact
def plot_data(mcat_name=mcat_name_list,unit_type=unit_type_list):
    if mcat_name+'_'+unit_type in cleaned_price.keys():
        x=cleaned_price[mcat_name+'_'+unit_type]
        plt.figure(figsize=[20,5])
        d=sns.distplot(x, bins=20, kde=True, rug=True,)
    

interactive(children=(Dropdown(description='mcat_name', options=('HMI Touch Panel', 'AC Motors', 'Single Phase…

In [67]:
@interact
def plot_data(mcat_name=mcat_name_list,unit_type=unit_type_list):
    if mcat_name+'_'+unit_type in cleaned_price.keys():
        x=cleaned_price[mcat_name+'_'+unit_type]
        plt.figure(figsize=[20,5])
        d=sns.distplot(x, bins=bayesian_blocks(x, fitness='events', p0=0.01), kde=True, rug=True,)
    

interactive(children=(Dropdown(description='mcat_name', options=('HMI Touch Panel', 'AC Motors', 'Single Phase…

### Using Dynamic Algo for finding the optimal binning value

In [29]:
def bayesian_blocks_f(t):
    # copy and sort the array
    t = np.sort(t)
    N = t.size

    # create length-(N + 1) array of cell edges
    edges = np.concatenate([t[:1],
                            0.5 * (t[1:] + t[:-1]),
                            t[-1:]])
    block_length = t[-1] - edges

    # arrays needed for the iteration
    nn_vec = np.ones(N)
    best = np.zeros(N, dtype=float)
    last = np.zeros(N, dtype=int)

    for K in range(N):
        # Compute the width and count of the final bin for all possible
        # locations of the K^th changepoint
        width = block_length[:K + 1] - block_length[K + 1]
        count_vec = np.cumsum(nn_vec[:K + 1][::-1])[::-1]

        # evaluate fitness function for these possibilities
        fit_vec = count_vec * (np.log(count_vec) - np.log(width))
        fit_vec -= 4  # 4 comes from the prior on the number of changepoints
        fit_vec[1:] += best[:K]

        # find the max of the fitness: this is the K^th changepoint
        i_max = np.argmax(fit_vec)
        last[K] = i_max
        best[K] = fit_vec[i_max]

    change_points =  np.zeros(N, dtype=int)
    i_cp = N
    ind = N
    while True:
        i_cp -= 1
        change_points[i_cp] = ind
        if ind == 0:
            break
        ind = last[ind - 1]
    change_points = change_points[i_cp:]

    return edges[change_points]

In [57]:
most_com1,most_com2,most_com3=0,0,0
three_most_common = {}
count = 0
for cat in cleaned_price:
    
    d={}
    
    edges = bayesian_blocks(cleaned_price[cat], fitness='events', p0=0.01)
    
    edges = list(map(int,edges))
    
    if len(edges)==1:
        most_com1=most_com2=most_com3=edges[0]
    
    else:
        
        for r in range(0,len(edges)-1):
            if r==0:
                d['0-'+str(edges[0])]=len(list(filter(lambda x: True if x in range(0,edges[0]) else False,cleaned_price[cat])))
                d[str(edges[r])+'-'+str(edges[r+1])] = len(list(filter(lambda x: False if x in range(0,edges[0]) else True,cleaned_price[cat])))
            else:

                d[str(edges[r-1])+'-'+str(edges[r])] = len(list(filter(lambda x: True if x in range(edges[r-1],edges[r]) else False,cleaned_price[cat])))
                d[str(edges[r])+'-'+str(edges[r+1])] = len(list(filter(lambda x: False if x in range(edges[r-1],edges[r]) else True,cleaned_price[cat])))

        d = sorted(d,key=lambda x: d[x],reverse=True)
        
        if len(d) == 2:
            
            most_com1 = np.mean([int(d[0].split('-')[0]),int(d[0].split('-')[1])])
            most_com2 = most_com3 = most_com1 = np.mean([int(d[1].split('-')[0]),int(d[1].split('-')[1])])
        
        elif len(d) != 1:
            
        
            most_com1 = np.mean([int(d[0].split('-')[0]),int(d[0].split('-')[1])])
            most_com2 = np.mean([int(d[1].split('-')[0]),int(d[1].split('-')[1])])
            most_com3 = np.mean([int(d[2].split('-')[0]),int(d[2].split('-')[1])])    
                    
    
    
    three_most_common[count]={'category':cat.split('_')[0],'unit_type':cat.split('_')[1],'MOST_COMMON_1':most_com1,'MOST_COMMON_2':most_com2,'MOST_COMMON_3':most_com3}
    count += 1
                
                
                
        
    

In [59]:
final_output_data_3=pd.DataFrame(three_most_common).T

In [68]:
final_output_data_3 = final_output_data_3[['category','unit_type','MOST_COMMON_1','MOST_COMMON_2','MOST_COMMON_3']]

In [69]:
final_output_data_3.to_csv('MCAT-Unit_wise--Three_most_common_prices.csv')

In [70]:
final_output_data_3

,category,unit_type,MOST_COMMON_1,MOST_COMMON_2,MOST_COMMON_3
0,AC Motors,box,1600,1600,1600
1,AC Motors,number,11000,11000,11000
2,AC Motors,piece,4512.5,8875,1661
3,AC Motors,set,100,100,100
4,AC Motors,unit,3750,16750,1000
5,Acme Safety Shoes,box,860,860,860
6,Acme Safety Shoes,pair,831,331,474.5
7,Acme Safety Shoes,piece,100,100,100
8,Acme Safety Shoes,unit,340,340,340
9,Air Conditioner Voltage Stabilizer,box,150,150,150
